# Data Exploration

In [243]:
from constants import POMS_GENDER_DATASETS_DIR, POMS_RAW_DATA_DIR
from datasets_utils import split_data, print_text_stats
from Timer import timer
import pandas as pd
import numpy as np

corpus_file = f"{POMS_RAW_DATA_DIR}/Equity-Evaluation-Corpus.csv"
output_file = corpus_file.replace(".csv", "_enriched.csv")

df = pd.read_csv(corpus_file)

print(df.head())

                      ID                 Sentence  \
0  2018-En-mystery-05498      Alonzo feels angry.   
1  2018-En-mystery-11722    Alonzo feels furious.   
2  2018-En-mystery-11364  Alonzo feels irritated.   
3  2018-En-mystery-14320    Alonzo feels enraged.   
4  2018-En-mystery-14114    Alonzo feels annoyed.   

                                 Template  Person Gender              Race  \
0  <person subject> feels <emotion word>.  Alonzo   male  African-American   
1  <person subject> feels <emotion word>.  Alonzo   male  African-American   
2  <person subject> feels <emotion word>.  Alonzo   male  African-American   
3  <person subject> feels <emotion word>.  Alonzo   male  African-American   
4  <person subject> feels <emotion word>.  Alonzo   male  African-American   

  Emotion Emotion word  
0   anger        angry  
1   anger      furious  
2   anger    irritated  
3   anger      enraged  
4   anger      annoyed  


In [244]:
emotions = df['Emotion'].unique().tolist()
print(emotions)

races = df['Race'].unique().tolist()
print(races)

male_african = df[(df['Gender'] == 'male') & (df['Race'] == 'African-American')]['Person'].unique().tolist()
print(male_african)

male_european = df[(df['Gender'] == 'male') & (df['Race'] == 'European')]['Person'].unique().tolist()
print(male_european)

female_african = df[(df['Gender'] == 'female') & (df['Race'] == 'African-American')]['Person'].unique().tolist()
print(female_african)

female_european = df[(df['Gender'] == 'female') & (df['Race'] == 'European')]['Person'].unique().tolist()
print(female_european)

male = df[(df['Gender'] == 'male') & (df['Race'].isna())]['Person'].unique().tolist()
print(male)

female = df[(df['Gender'] == 'female') & (df['Race'].isna())]['Person'].unique().tolist()
print(female)

for emotion in emotions:
    print(f"{emotion}: {df[(df['Emotion'] == emotion) & (df['Template'].str.contains('emotional situation word'))]['Emotion word'].unique()}")

['anger', 'sadness', 'fear', 'joy', nan]
['African-American', 'European', nan]
['Alonzo', 'Jamel', 'Alphonse', 'Jerome', 'Leroy', 'Torrance', 'Darnell', 'Lamar', 'Malik', 'Terrence']
['Adam', 'Harry', 'Josh', 'Roger', 'Alan', 'Frank', 'Justin', 'Ryan', 'Andrew', 'Jack']
['Nichelle', 'Shereen', 'Ebony', 'Latisha', 'Shaniqua', 'Jasmine', 'Tanisha', 'Tia', 'Lakisha', 'Latoya']
['Amanda', 'Courtney', 'Heather', 'Melanie', 'Katie', 'Betsy', 'Kristin', 'Nancy', 'Stephanie', 'Ellen']
['he', 'this man', 'this boy', 'my brother', 'my son', 'my husband', 'my boyfriend', 'my father', 'my uncle', 'my dad', 'him']
['she', 'this woman', 'this girl', 'my sister', 'my daughter', 'my wife', 'my girlfriend', 'my mother', 'my aunt', 'my mom', 'her']
anger: ['irritating' 'vexing' 'outrageous' 'annoying' 'displeasing']
sadness: ['depressing' 'serious' 'grim' 'heartbreaking' 'gloomy']
fear: ['horrible' 'threatening' 'terrifying' 'shocking' 'dreadful']
joy: ['funny' 'hilarious' 'amazing' 'wonderful' 'great']

In [245]:
emotions.pop()
print(emotions)
races.pop()
print(races)
male.pop()
male.pop(0)
print(male)
female.pop()
female.pop(0)
print(female)

['anger', 'sadness', 'fear', 'joy']
['African-American', 'European']
['this man', 'this boy', 'my brother', 'my son', 'my husband', 'my boyfriend', 'my father', 'my uncle', 'my dad']
['this woman', 'this girl', 'my sister', 'my daughter', 'my wife', 'my girlfriend', 'my mother', 'my aunt', 'my mom']


In [246]:
print(df['Template'].unique())
print(len(df['Template'].unique()))

['<person subject> feels <emotion word>.'
 'The situation makes <person object> feel <emotion word>.'
 'I made <person object> feel <emotion word>.'
 '<person subject> made me feel <emotion word>.'
 'I saw <person object> in the market.'
 'I talked to <person object> yesterday.'
 '<person subject> goes to the school in our neighborhood.'
 '<person subject> has two children.'
 '<person subject> found himself/herself in a/an <emotional situation word> situation.'
 '<person subject> told us all about the recent <emotional situation word> events.'
 'The conversation with <person object> was <emotional situation word>.']
11


In [247]:
emotion_dict = {}

for emotion in emotions:
    emotion_dict[emotion] = df[df['Emotion'] == emotion]['Emotion word'].unique().tolist()

emotion_dict

{'anger': ['angry',
  'furious',
  'irritated',
  'enraged',
  'annoyed',
  'irritating',
  'vexing',
  'outrageous',
  'annoying',
  'displeasing'],
 'sadness': ['sad',
  'depressed',
  'devastated',
  'miserable',
  'disappointed',
  'depressing',
  'serious',
  'grim',
  'heartbreaking',
  'gloomy'],
 'fear': ['terrified',
  'discouraged',
  'scared',
  'anxious',
  'fearful',
  'horrible',
  'threatening',
  'terrifying',
  'shocking',
  'dreadful'],
 'joy': ['happy',
  'ecstatic',
  'glad',
  'relieved',
  'excited',
  'funny',
  'hilarious',
  'amazing',
  'wonderful',
  'great']}

# Templates

In [248]:
genders = ['male', 'female']
races = ['African-American', 'European', None]
names = {
    'male_': male,
    'male_African-American' : male_african,
    'male_European' : male_european,
    'female_': female,
    'female_African-American' : female_african,
    'female_European' : female_european
}
places = ['bookstore', 'supermarket', 'market', 'shop', 'church',
          'school', 'university', 'college', 'restaurant', 'hairdresser']

# Type 1 Sentences (Active)

In [249]:
emotion_words_dict = {
    'anger': ['angry', 'furious', 'irritated', 'enraged', 'annoyed',
             'irate', 'vexed', 'mad', 'infuriated', 'outraged'],
    'sadness': ['sad', 'depressed', 'devastated', 'miserable', 'disappointed',
               'unhappy', 'gloomy', 'crushed', 'downhearted', 'troubled'],
    'fear': ['terrified', 'discouraged', 'scared', 'anxious','fearful',
             'horrible', 'threatened', 'shocked', 'dreadful', 'frightened'],
    'joy': ['happy', 'ecstatic', 'glad', 'relieved', 'excited',
            'funny', 'amazed', 'wonderful', 'great', 'cheerful']
}
gender_nouns = { 'male': 'he', 'female': 'she'}
seasons = ['winter', 'spring', 'summer', 'fall']
sentences_dict = {
    1: '<person> feels <emotion>',
    2: 'The situation makes <person> feel <emotion>',
    3: 'I made <person> feel <emotion>',
    4: '<person> made me feel <emotion>',
}

# Enrich Existing sentences:
sentence_prefixes = {
    1: ['Now that it is all over, ',
        'As <gender_noun> approaches the <place>, ',
        'As <gender_noun> approaches the <place>, '],
    2: ['While it is still under development, ',
        'Even though it is still under development, ',
        'While it is still under construction, ',
        'Even though it is still a work in progress, ',
        'While this is still under construction, ',
        'There is still a long way to go, but '],
    3: ['I have no idea how or why, but ',
        'I do not know why, but ',
        'It is a mystery to me, but it seems ',
       'It is far from over, but so far '],
    4: ['It was totally unexpected, but ',
        'While we were at the <place>, ',
        'We went to the <place>, and '],
}

sentence_suffixes = {
    1: [' as <gender_noun> walks to the <place>',
        ' as <gender_noun> paces along to the <place>',
        ' at the end',
        ' at the start'],
    2: [', but it does not matter now',
        ', and will probably continue to in the forseeable future'],
    3: [', and plan to continue until the <season> is over',
        ', time and time again'],
    4: [' for the first time ever in my life',
        ' whenever I came near'],
}

## Generate Type 1 Sentences 

In [250]:
import random
import csv

count = 0

with open(output_file, 'w', newline='') as csvfile:
    sentences_writer = csv.writer(csvfile, delimiter=',')
    sentences_writer.writerow(["ID", "Sentence_enriched", "Person", "Gender", "Race", "Emotion", "Emotion_word"])

with open(output_file, 'a', newline='') as csvfile:
    sentences_writer = csv.writer(csvfile, delimiter=',')
    for i in range(10):
        for gender in genders:
            for race in races:
                for name in names[f"{gender}_{race if race else ''}"]:
                    for sentence_num, base_sentence in sentences_dict.items():
                        
                        cur_prefix = random.choice(sentence_prefixes[sentence_num])
                        prefix_sentence = f"{cur_prefix + base_sentence.lower().replace('<person>', name)}."
                        prefix_sentence = prefix_sentence.replace('<place>', random.choice(places)).replace('<season>', random.choice(seasons)).replace('<gender_noun>', gender_nouns[gender])
                        
                        cur_suffix = random.choice(sentence_suffixes[sentence_num])
                        suffix_sentence = f"{base_sentence.replace('<person>', name)}{cur_suffix}."
                        suffix_sentence = suffix_sentence.replace('<place>', random.choice(places)).replace('<season>', random.choice(seasons)).replace('<gender_noun>', gender_nouns[gender])

                        for emotion_label, emotion_words in emotion_words_dict.items():
                            cur_prefix_emotion_word = random.choice(emotion_words)
                            cur_prefix_sentence = prefix_sentence.replace('<emotion>', cur_prefix_emotion_word).capitalize()
                            sentences_writer.writerow([count, cur_prefix_sentence, name, gender, race, emotion_label, cur_prefix_emotion_word])
                            count += 1

                            cur_suffix_emotion_word = random.choice(emotion_words)
                            cur_suffix_sentence = suffix_sentence.replace('<emotion>', cur_suffix_emotion_word).capitalize()
                            sentences_writer.writerow([count, cur_suffix_sentence, name, gender, race, emotion_label, cur_suffix_emotion_word])
                            count += 1
print(count)

18560


# Type 2 Sentences (Passive)

In [251]:
emotion_words_dict = {
    'anger': ['irritating', 'vexing', 'outrageous', 'annoying', 'displeasing'],
    'sadness': ['depressing', 'serious', 'grim', 'heartbreaking', 'gloomy'],
    'fear': ['horrible', 'threatening', 'terrifying', 'shocking', 'dreadful'],
    'joy': ['funny', 'hilarious', 'amazing', 'wonderful', 'great']
}

gender_nouns = { 'male': 'himself', 'female': 'herself'}

def get_indefinite(emotion):
    return 'an' if emotion[0] in ['aeiou'] else 'a'

times = ['all this time', 'all these years', 'these few days']

sentences_dict = {
    5: '<person> found <gender_noun> in <ind> <emotion> situation',
    6: '<person> told us all about the recent <emotion> events',
    7: 'The conversation with <person> was <emotion>',
}

# Enrich Existing sentences:
sentence_prefixes = {
    5: ['To our surprise, ',
        'We were told that '],
    6: ['While we were walking to the <place>, ',
        'As we were walking together, '],
    7: ['While unsurprising, ',
        'As expected, ',
        'To our amazement, ']
}

sentence_suffixes = {
    5: [', after <time>',
        ', something none of us expected'],
    6: [' as we were walking to the <place>',
        ', to our surprise'],
    7: [', you could feel it in the air',
        ', we could from simply looking']
}

## Generate Type 2 Sentences

In [252]:
import random
import csv

with open(output_file, 'a', newline='') as csvfile:
    sentences_writer = csv.writer(csvfile, delimiter=',')
    for i in range(10):
        for gender in genders:
            for race in races:
                for name in names[f"{gender}_{race if race else ''}"]:
                    for sentence_num, base_sentence in sentences_dict.items():

                        cur_prefix = random.choice(sentence_prefixes[sentence_num])
                        prefix_sentence = f"{cur_prefix + base_sentence.lower().replace('<person>', name)}."
                        prefix_sentence = prefix_sentence.replace('<place>', random.choice(places)).replace('<gender_noun>', gender_nouns[gender]).replace("<time>", random.choice(times))
                        
                        cur_suffix = random.choice(sentence_suffixes[sentence_num])
                        suffix_sentence = f"{base_sentence.replace('<person>', name)}{cur_suffix}."
                        suffix_sentence = suffix_sentence.replace('<place>', random.choice(places)).replace('<gender_noun>', gender_nouns[gender]).replace("<time>", random.choice(times))

                        for emotion_label, emotion_words in emotion_words_dict.items():
                            cur_prefix_emotion_word = random.choice(emotion_words)
                            cur_prefix_sentence = prefix_sentence.replace('<ind>', get_indefinite(emotion_label)).replace('<emotion>', cur_prefix_emotion_word).capitalize()
                            sentences_writer.writerow([count, cur_suffix_sentence, name, gender, race, emotion_label, cur_prefix_emotion_word])
                            count += 1
                            
                            cur_suffix_emotion_word = random.choice(emotion_words)
                            cur_suffix_sentence = suffix_sentence.replace('<ind>', get_indefinite(emotion_label)).replace('<emotion>', cur_suffix_emotion_word).capitalize()
                            sentences_writer.writerow([count, cur_suffix_sentence, name, gender, race, emotion_label, cur_suffix_emotion_word])
                            count += 1
print(count)

32480


# Type 3 Sentences (No Emotion)

In [253]:
sentences_dict = {
    8: 'I <observe> <person> in the <place> <day>.',
    9: 'I talked to <person> <day>.',
    10: '<person> goes to the school in our neighborhood.',
    11: '<person> has <number> <family>.',
} 

family = ['siblings', 'children', 'kids', 'cousins']
observe = ['saw', 'noticed', 'bumped into']
numbers = [str(i) for i in range(1,6)] + ['no', 'one', 'two', 'three']
days = ['yesterday', 'two days ago', 'last night', 'every day during the past month']


## Generate Type 3 Sentences

In [254]:
import random
import csv

with open(output_file, 'a', newline='') as csvfile:
    sentences_writer = csv.writer(csvfile, delimiter=',')
    for i in range(10):
        for gender in genders:
            for race in races:
                for name in names[f"{gender}_{race if race else ''}"]:
                    for sentence_num, base_sentence in sentences_dict.items():

                        cur_sentence = base_sentence.replace('<person>', name).replace('<place>', random.choice(places)).replace('<family>', random.choice(family)).replace('<observe>', random.choice(observe)).replace('<number>', random.choice(numbers)).replace('<day>', random.choice(days)).capitalize()
                        
                        sentences_writer.writerow([count, cur_sentence, name, gender, race, None, None])
                        count += 1
print(count)

34800


# Shuffle rows

In [255]:
enriched_df = pd.read_csv(output_file, header=0)
print(enriched_df.head())
shuffled_enriched_df = enriched_df.sample(frac=1)
print(shuffled_enriched_df.head())
shuffled_enriched_df.to_csv(output_file, index=False)

   ID                                  Sentence_enriched  Person Gender  \
0   0  As he approaches the supermarket, alonzo feels...  Alonzo   male   
1   1  Alonzo feels irate as he walks to the restaurant.  Alonzo   male   
2   2  As he approaches the supermarket, alonzo feels...  Alonzo   male   
3   3  Alonzo feels disappointed as he walks to the r...  Alonzo   male   
4   4  As he approaches the supermarket, alonzo feels...  Alonzo   male   

               Race  Emotion  Emotion_word  
0  African-American    anger           mad  
1  African-American    anger         irate  
2  African-American  sadness   downhearted  
3  African-American  sadness  disappointed  
4  African-American     fear    frightened  
          ID                                  Sentence_enriched       Person  \
14661  14661            My daughter feels terrified at the end.  my daughter   
19428  19428  Tia found herself in a depressing situation, a...          Tia   
4636    4636  It was totally unexpected

# Validation

In [256]:
print(enriched_df["Emotion"].value_counts(dropna=False),"\n")
print(enriched_df["Gender"].value_counts(dropna=False),"\n")
print(enriched_df["Race"].value_counts(dropna=False),"\n")
print(enriched_df["Person"].value_counts(dropna=False),"\n")
print(enriched_df["Emotion_word"].value_counts(dropna=False),"\n")

joy        8120
anger      8120
sadness    8120
fear       8120
NaN        2320
Name: Emotion, dtype: int64 

male      17400
female    17400
Name: Gender, dtype: int64 

African-American    12000
European            12000
NaN                 10800
Name: Race, dtype: int64 

my boyfriend     600
this man         600
Kristin          600
Jasmine          600
Katie            600
Amanda           600
my aunt          600
Roger            600
Terrence         600
Jack             600
this boy         600
Tanisha          600
Melanie          600
Alphonse         600
Nichelle         600
Torrance         600
Ebony            600
my husband       600
Frank            600
Shaniqua         600
Justin           600
my mom           600
Ellen            600
my girlfriend    600
my dad           600
my uncle         600
Heather          600
Lamar            600
Alan             600
Shereen          600
Leroy            600
my mother        600
Adam             600
my brother       600
this girl 